**HIGH LEVEL ANALYSIS**
===========


In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import straxen
from multihist import Histdd, Hist1d

# Print out exactly what versions we are using, for reference / troubleshooting:
import sys
import os.path as osp
print(f"Python {sys.version} at {sys.executable}\n"
      f"Straxen {straxen.__version__} at {osp.dirname(straxen.__file__)}")

Python 3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) 
[GCC 7.3.0] at /opt/XENONnT/anaconda/envs/XENONnT_development/bin/python
Straxen 0.9.0 at /opt/XENONnT/anaconda/envs/XENONnT_development/lib/python3.6/site-packages/straxen


In [2]:
st = straxen.contexts.xenon1t_dali()
run_id = '180215_1029'

In [3]:
peaks = st.get_array(run_id,['peaks','peak_basics'])

**PEAK PROCESSING: peak_processing.py**
=====

**Peak Basics**
--------------

Stiamo guardando il codice del plugin peak_processing


Domanda sulla funzione compute, prende dei piccoli chunck di picchi e prende peaks che è un array, compute_center_times si potrebbe migliorare, si potrebbe implementare diversi modi per calcolarlo.

In [4]:
#Per vedere le opzioni dei picchi:
st.show_config('peak_basics')

,option,default,current,applies_to,help
0,n_top_pmts,253,127,"(peak_basics,)",Number of top PMTs
1,diagnose_sorting,False,<OMITTED>,"(peaks,)",Enable runtime checks for sorting and disjoint...
2,peaklet_gap_threshold,350,<OMITTED>,"(peaklets, lone_hits)",No hits for this many ns triggers a new peak
3,peak_left_extension,30,<OMITTED>,"(peaklets, lone_hits)",Include this many ns left of hits in peaks
4,peak_right_extension,200,30,"(peaklets, lone_hits)",Include this many ns right of hits in peaks
5,peak_min_pmts,4,2,"(peaklets, lone_hits)",Minifnmum contributing PMTs needed to define a...
6,peak_split_gof_threshold,"(None, ((0.5, 1), (3.5, 0.25)), ((2, 1), (4.5,...",<OMITTED>,"(peaklets, lone_hits)",Natural breaks goodness of fit/split threshold...
7,peak_split_filter_wing_width,70,<OMITTED>,"(peaklets, lone_hits)",Wing width of moving average filter for low-sp...
8,peak_split_min_area,40,<OMITTED>,"(peaklets, lone_hits)",Minimum area to evaluate natural breaks criter...
9,peak_split_iterations,20,<OMITTED>,"(peaklets, lone_hits)",Maximum number of recursive peak splits to do.


In [5]:
st.data_info('peak_basics')

,Field name,Data type,Comment
0,time,int64,Start time of the peak (ns since unix epoch)
1,endtime,int64,End time of the peak (ns since unix epoch)
2,center_time,int64,Weighted center time of the peak (ns since uni...
3,area,float32,Peak integral in PE
4,n_channels,int16,Number of PMTs contributing to the peak
5,max_pmt,int16,PMT number which contributes the most PE
6,max_pmt_area,float32,Area of signal in the largest-contributing PMT...
7,range_50p_area,float32,Width (in ns) of the central 50% area of the peak
8,range_90p_area,float32,Width (in ns) of the central 90% area of the peak
9,area_fraction_top,float32,Fraction of area seen by the top array (NaN fo...


Calcolo di **range_50p_area** e **range_90p_area**, viene presa la larghezza del picco e poi usato :,

In [6]:
print(peaks['width'][:,5])
print(peaks['width'][:,9])

[196.87552  83.1763  231.50201 ... 383.8594  268.03873 167.85959]
[ 208.07713   97.32471  246.52963 ... 3063.1973   275.9225   195.40831]


Calcolo di **area_fraction_top**

In [7]:
n_top = st.config['n_top_pmts']
area_top = peaks['area_per_channel'][:, :n_top].sum(axis=1)
m = peaks['area'] > 0
#r['area_fraction_top'][m] = 
area_top[m]/peaks['area'][m]

array([0.4595095 , 0.36514154, 0.        , ..., 0.4225748 , 1.        ,
       1.        ], dtype=float32)

Calcolo di **rise_time**

In [8]:
-peaks['area_decile_from_midpoint'][:,1]

array([ 14.280781,  86.96509 ,  16.90953 , ..., 234.96863 ,  39.32416 ,
       186.01207 ], dtype=float32)


**Peak Positions**
-------------------

La ricostruzione della posizione, prende la somma totale di più PMT inseme, ma non considera la posizione dei singoli PMT.

Al momento la posizione dei picchi è ricostruita usando un solo metodo, si può introdurre un nuovo algoritmo per fare questo.

In [9]:
st.data_info('peak_positions')

,Field name,Data type,Comment
0,x,float32,"Reconstructed S2 X position (cm), uncorrected"
1,y,float32,"Reconstructed S2 Y position (cm), uncorrected"
2,time,int64,Start time since unix epoch [ns]
3,endtime,int64,Exclusive end time since unix epoch [ns]


In [10]:
st.register(straxen.plugins.peak_processing.PeakPositions)

straxen.plugins.peak_processing.PeakPositions

Viene caricato un file json dove sono inseriti i PMT non funzionanti, poi vengono presi soltanto i PMT che hanno un'area del picco maggiore di un certo valore definendo un mask e poi viene usata la funzione predict per trovare la posizione.

**Peak Proximity**
-----------------

L'obiettivo è quello di controllare picchi nelle vicinanze, si può cambiare la finestra che si guarda con get_window_size con OverlapWindowPlugin, guardare qui per più dettagli https://strax.readthedocs.io/en/latest/developer/overlaps.html#overlap-window-plugins.


In [11]:
st.data_info('peak_proximity')

,Field name,Data type,Comment
0,n_competing,int32,Number of nearby larger or slightly smaller peaks
1,n_competing_left,int32,Number of larger or slightly smaller peaks lef...
2,t_to_prev_peak,int64,Time between end of previous peak and start of...
3,t_to_next_peak,int64,Time between end of this peak and start of nex...
4,t_to_nearest_peak,int64,Smaller of t_to_prev_peak and t_to_next_peak [ns]
5,time,int64,Start time since unix epoch [ns]
6,endtime,int64,Exclusive end time since unix epoch [ns]


**EVENT PROCESSING: event_processing.py**
===========================


Ci sono 5 classi nel codice e alla fine c'è **Event Info** con :
1. Events
2. EventsBasics
3. EventsPositions
4. CorrectedAreas
5. EnergyEstimate

**Events**
---------------

Dipende da peak_basics e peak_proximity. Ci sono argomenti start e end, che se usati vengono presi come intervalli nel quale cercare gli eventi (può essere utile se ad esempio ci sono dei periodi in cui non ho dati).


In [12]:
st.data_info('events')

,Field name,Data type,Comment
0,event_number,int64,Event number in this dataset
1,time,int64,Event start time in ns since the unix epoch
2,endtime,int64,Event end time in ns since the unix epoch


Il trigger è cercato con un minimo (100) è un massimo.


**Events Basics**
----------------------

Dipende da events, peak_basics, peak_positions, peak_proximity e calcola le proprietà di base per ogni evento:

In [13]:
st.data_info('event_basics')

,Field name,Data type,Comment
0,time,int64,Start time since unix epoch [ns]
1,endtime,int64,Exclusive end time since unix epoch [ns]
2,n_peaks,int32,Number of peaks in the event
3,drift_time,int32,Drift time between main S1 and S2 in ns
4,s1_index,int32,Main S1 peak index in event
5,alt_s1_index,int32,Alternate S1 peak index in event
6,s1_time,int64,Main S1 start time since unix epoch [ns]
7,alt_s1_time,int64,Alternate S1 start time since unix epoch [ns]
8,s1_center_time,int64,Main S1 weighted center time since unix epoch ...
9,alt_s1_center_time,int64,Alternate S1 weighted center time since unix e...


Calcola le proprietà facendo un loop sui 2 picchi (S1 e S2), c'è anche la possibilità di fare il loop 

In [ ]:
runs = st.select_runs(run_mode='kr*', available='event_info')
run_id = '181027_2044'
st.is_stored(run_id,'event_info')
events = st.get_df(run_info['name'], 'event_info')
events = st.get_df(run_info['name'], 'event_info')